<img src='op2-u02.png'/>
<h2><font color='#7F0000'>OP2-09-Aplicação com Banco de Dados</font></h2>

<table width='100%'>
    <tbody>
        <tr>
            <td width='33%' style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>Notebook Anterior<br><a href="OP2-08-Banco-de-Dados.ipynb">OP2-08-Banco-de-Dados</a></td>
            <td width='34%' style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>&nbsp;<br/>
            </td>
            <td width='33%'style='text-align: left; background-color: #DDDDDD; vertical-align: top;'>Próximo Notebook<br/><a href="../Notebooks%20U03/OP2-10-OO-Classes-e-Objetos.ipynb">OP2-10-OO-Classes-e-Objetos</a></td>
        </tr>
    </tbody>
</table>

## Pneus e suas aplicações

O uso de veículos terrestres, como automóveis, motocicletas, ônibus e caminhões, é enorme e essencial tanto do ponto de vista da mobilidade, como sob os aspectos econômicos. Um dos aspectos comuns à todos os veículos terrestre é o uso de pneus.

Com o uso (e até mesmo o passar do tempo) é necessário substituir os pneus desgastados e velhos dos veículos por outros novos, sempre com as mesmas características especificadas pelos fabricantes dos veículos.

Dado o enorme mercado para a venda de pneus, os fabricantes existentes oferecem vários modelos para atender uma grande gama de aplicações, de maneira que podem existir modelos de pneus diferentes capazes de satisfazer as necessidades de um veículo.

Concentrando nossa atenção nos automóveis (de passeio, de transporte de passageiros e de carga), sabemos que as informações necessárias para caracterizar um pneu estão ilustradas na figura que segue.

<img src="op2-09-pneu-medidas-padrao.jpg" alt="Características dos pneus" />

Estas características são:

- Largura da **Banda** de rodagem, indicada em milímetros (mm);
- Proporção da **Altura** do pneu em relação à sua Largura, indicada em porcentagem (%);
- **Tipo** da construção do pneu, indicada com uma ou duas letras (por exemplo, 'R' para pneu de construção radial);
- **Diâmetro** interno do pneu (que corresponde ao diâmetro externo da roda), indicada em polegadas (pol);
- Índice de **Carga**, indicada como um número padronizado;
- Índice de **Velocidade**, indicada como uma ou duas letras padronizadas.

Assim, pode ser interessante dispor de um banco de dados capaz de catalogar os *fabricantes de pneus*, os *modelos de pneus* que oferecem, além de sua *aplicação* para diferentes veículos dos diversos *fabricantes de veículos* existentes.

## Importação do SQLite

Construiremos uma aplicação simples, para gerenciar um catálogo de aplicação de modelos de pneus, cujo banco de dados será suportando pelo *SQLite*, o que requer a importação de seu módulo <tt>sqlite3</tt>.

In [ ]:
# importação do módulo SQLite3
import sqlite3

Para utilizar comandos SQL em um banco de dados *SQLite* é necessário uma *conexão* com o banco de dados desejado e, também, um *cursor*, por meio do qual são executados os comandos SQL.

Antecipando as necessidades da aplicação pretendida, definimos a função que segue, capaz de retornar uma tupla contendo uma conexão válida e um cursor para o banco de dados indicado. Caso seja fornecida, opcionalmente, uma conexão, esta é utilizada para obter um novo cursor, ou substituída caso inválida.

In [22]:
def get_cursor(database, connection=None):
    '''Retorna conexão e cursor para operação no .'''
    try:
        cursor = connection.cursor()
    except:
        # Obtém conexão, que cria banco de dados caso não exista
        connection = sqlite3.connect(database)
        cursor = connection.cursor()
    return (connection, cursor)

Uma maneira simples de utilizar a função <tt>get_cursor</tt> é fornecendo apenas a <tt>url</tt> do banco de dados. Observe que esta função retorna uma tupla contendo a conexão e o respectivo cursor.

In [23]:
resultado = get_cursor('db/metereologia.db')
resultado

(<sqlite3.Connection at 0x2869e82c6c0>, <sqlite3.Cursor at 0x2869e90bab0>)

A conexão e o cursor podem ser recupadados pela indexação dos elementos da tupla:

In [24]:
con = resultado[0]
cur = resultado[1]
print(con, cur)

<sqlite3.Connection object at 0x000002869E82C6C0> <sqlite3.Cursor object at 0x000002869E90BAB0>


Outra maneira mais simples de utilizar os resultados retornados por <tt>get_cursor</tt> é empregar o desempacotamento implícito de tuplas:

In [25]:
con, cur = get_cursor('db/metereologia.db')
print(con, cur)

<sqlite3.Connection object at 0x000002869E82C210> <sqlite3.Cursor object at 0x000002869E860F10>


Para criar apenas um cursor a partir de uma conexão existente, também é conveniente fazer:

In [26]:
_, cur = get_cursor('db/metereologia.db', con)
print(con, cur)

<sqlite3.Connection object at 0x000002869E82C210> <sqlite3.Cursor object at 0x000002869E911420>


## Criação do Banco de Dados Pneu

A função que segue, <tt>create_database</tt> permite a criação ou recriação do banco de dados <tt>Pneu</tt>.

Os comandos SQL para criação do banco de dados foram organizados num arquivo texto, permitindo adequações no *schema* desejado para o banco de dados, sem necessidade de modificação do programa. Para permitir a recriação, as tabelas são previamente removidas.

A captura de exceção na execução individual de cada comendo SQL recuperado do arquivo evita interromper a função quando uma ou mais tabelas não existem. 

In [ ]:
def create_database():
    '''Cria ou recria o banco de dados Pneus, retornando uma conexão para o mesmo.'''
    # Obtém cursor
    conexao, cursor = get_cursor('db/pneu.db')
    
    # Obtém comandos para criação do BD à partir do arquivo indicado
    arquivo = open('db/opy2_pneu_db_sqlite_create.sql', 'r')
    conteudo = arquivo.read()
    arquivo.close()
    # Conteúdo é dividido em comandos separados por ponto-e-vírgula
    comandos = conteudo.split(';')
    # Execução de comando SQL com cursor pré-definido
    for cmd in comandos:
        cmd = cmd.strip().strip('\n\n').strip('\n')
        if len(cmd) == 0:
            continue # pula linhas em branco
        try:
            print('.', cmd) # exibe comando recuperado
            cursor.execute(cmd) # executa comando recuperado
        except Exception as exc:
            print(exc)
            
    # SQL DTL (consolidação de operações)
    conexao.commit()
    # Retorna conexão em uso
    return conexao

O acionamento desta função permite criar o banco de dados <tt>Pneu</tt>, como segue.

In [ ]:
# Efetua a criação do banco de dados e
# retém sua conexão para uso posterior
conexao = create_database()

O banco de dados criado possui a seguinte estrutura.

<img src="op2-09-pneus-db.png" alt="BD Pneu" />

A estrutura do banco de dados pode ser conferida como uso de um visualizador, tal como o *DB Browser for SQLite*.

https://sqlitebrowser.org/

## Inclusão de Fabricantes de Pneu

Para incluir fabricantes de pneu no banco de dados <tt>Pneu</tt> é conveniente uma função que realize esta tarefa, tomando apenas os dados necessários, que neste caso é apenas o *nome do fabricante de pneus*, e, opcionalmente, uma conexão para o banco de dados, efetuando a inclusão do fabricante de pneus no banco de dados.

In [ ]:
def insert_fab_pneu(nome_fab_pneu, connection=None):
    '''Efetua a inclusão de fabricante na tabela Fab_Pneu.'''
    # Obtém conexão e cursor
    connection, cursor = get_cursor('db/pneu.db', connection)
    # SQL para inserção de fabricante de pneu
    sql = f"INSERT INTO Fab_Pneu(nome) VALUES('{nome_fab_pneu}')"
    # Efetua inserção
    cursor.execute(sql)
    # SQL DTL (consolidação de operações)
    connection.commit()

A função <tt>insert_fab_pneu</tt> pode ser usada simplesmente com a indicação do nome do fabricante de pneus.

In [ ]:
# Inserção de fabricantes de pneu
insert_fab_pneu('Pirelli', conexao)
insert_fab_pneu('Goodyear', conexao)
insert_fab_pneu('Michelin', conexao)
insert_fab_pneu('Firestone', conexao)
insert_fab_pneu('Continental', conexao)

Na seção que segue construiremos uma função para a consulta dos fabricantes de pneus cadastrados no banco de dados <tt>Pneu</tt>.

## Consulta de Fabricante de Pneus

Para consultar os fabricantes de pneus no banco de dados <tt>Pneu</tt> é igualmente adequada uma função para realização desta tarefa, tomando opcionalmente uma conexão para o banco de dados. 

In [ ]:
def query_fab_pneu(connection=None):
    '''Função que efetua e exibe fabricantes de pneus cadastrados.'''
    # Obtém apenas cursor
    _, cursor = get_cursor('db/pneu.db', connection)
    # SQL para consulta de fabricante de pneu
    sql = 'SELECT * FROM Fab_Pneu ORDER BY Nome'
    # Efetua consulta
    cursor.execute(sql)
    # Dados da consulta realizada devem ser recuperados
    dados = cursor.fetchall()
    # Exibição dos dados consultados de maneira tabular
    print('+-%31s-+' % (31*'-'))
    print('| %-31s |' % ('FAB_PNEU'))
    print('+-%31s-+' % (31*'-'))
    print('| %-3s | %-25s |' % ('Cód', 'Nome'))
    print('+-%31s-+' % (31*'-'))
    for registro in dados:
        print('| %3d | %-25s |' % registro)
    print('+-%31s-+' % (31*'-'))
    print(len(dados), 'registros.')
    return

É bastante direto o uso da função <tt>query_fab_pneu</tt>.

In [ ]:
# Mostra dados da tabela Fab_Pneu
query_fab_pneu(conexao)

A título de ilustração, a figura que segue mostra o uso do 'DB Browser for SQLite* na visualização do conteúdo da tabela <tt>Fab_Pneu</tt>.

<img src="op2-09-db-browser-pneu-db.png" alt="DB Browser for SQLite" />

## Inclusão de Fabricantes de Veículos

A inclusão de fabricantes de veículos no banco de dados <tt>Pneu</tt> também pode ser feita por meio de uma função, que tome o único dado necessário, o *nome do fabricante de veículos*, e, opcionalmente, uma conexão para o banco de dados, efetuando a inclusão do fabricante de veículos no banco de dados.

In [ ]:
def insert_fab_veiculo(nome_fab_veiculo, connection=None):
    '''Efetua a inclusão de fabricante na tabela Fab_Veic.'''
    # Obtém conexão e cursor
    connection, cursor = get_cursor('db/pneu.db', connection)
    # SQL para inserção de fabricante de veículo
    sql = f"INSERT INTO Fab_Veic(nome) VALUES('{nome_fab_veiculo}')"
    # Efetua inserção
    cursor.execute(sql)
    # SQL DTL (consolidação de operações)
    connection.commit()

O uso da função <tt>insert_fab_veiculo</tt> é bastante direto e requer apenas a indicação do nome do fabricante de veículos.

In [ ]:
# Inserção de fabricantes de veículos
insert_fab_veiculo('Volkswagen', conexao)
insert_fab_veiculo('Fiat', conexao)
insert_fab_veiculo('Chevrolet', conexao)
insert_fab_veiculo('Ford', conexao)
insert_fab_veiculo('Honda', conexao)
insert_fab_veiculo('Toyota', conexao)
insert_fab_veiculo('Kia', conexao)
insert_fab_veiculo('Hyundai', conexao)
insert_fab_veiculo('Jeep', conexao)
insert_fab_veiculo('Audi', conexao)
insert_fab_veiculo('BMW', conexao)

Na seção que segue construiremos uma função para a consulta dos fabricantes de veículos cadastrados no banco de dados <tt>Pneu</tt>.

## Consulta de Fabricante de Veículos

Para consultar os fabricantes de veículos no banco de dados <tt>Pneu</tt> é feita de maneira idêntica à consulta dos fabricantes de pneus, com uso de uma função que toma opcionalmente uma conexão para o banco de dados. 

In [ ]:
def query_fab_veic(connection=None):
    '''Função que efetua e exibe fabricantes de veículos cadastrados.'''
    # Obtém apenas cursor
    _, cursor = get_cursor('db/pneu.db', connection)
    # SQL para consulta de fabricante de veículos
    sql = 'SELECT * FROM Fab_Veic ORDER BY Nome'
    # Efetua consulta
    cursor.execute(sql)
    # Dados da consulta realizada devem ser recuperados
    dados = cursor.fetchall()
    # Exibição dos dados consultados de maneira tabular
    print('+-%31s-+' % (31*'-'))
    print('| %-31s |' % ('FAB_VEIC'))
    print('+-%31s-+' % (31*'-'))
    print('| %3s | %-25s |' % ('Cód', 'Nome'))
    print('+-%31s-+' % (31*'-'))
    for registro in dados:
        print('| %3d | %-25s |' % registro)
    print('+-%31s-+' % (31*'-'))
    print(len(dados), 'registros.')
    return

É direto o uso da função <tt>query_fab_veic</tt>.

In [ ]:
# Mostra dados da tabela Fab_Veic
query_fab_veic(conexao)

## Inclusão de Pneus

A inclusão de modelos de pneus no banco de dados <tt>Pneu</tt> requer diversos dados: *id* do fabricante, modelo, banda, altura, tipo, diâmetro, carga e velocidade. Desta forma é bastante adequada uma função que tome tais parâmetros e, opcionalmente, uma conexão para o banco de dados, efetuando a inclusão de um modelo de pneu no banco de dados.

In [ ]:
def insert_pneu(fab_id, modelo, banda, altura, tipo, diam, carga, veloc, connection=None):
    '''Efetua a inclusão de modelo de pneu na tabela Pneu.'''
    # Obtém conexão e cursor
    connection, cursor = get_cursor('db/pneu.db', connection)
    # SQL para inserção de modelos de pneus
    sql = f"""INSERT INTO 
    Pneu(Fab_Id, modelo, banda, altura, tipo, diametro, carga, velocidade) 
    VALUES({fab_id}, '{modelo}', {banda}, {altura}, '{tipo}', {diam}, {carga}, '{veloc}')"""
    # Efetua inserção
    cursor.execute(sql)
    # SQL DTL (consolidação de operações)
    connection.commit()

O uso da função <tt>insert_pneu</tt> requer vários dados, como mostrado a seguir.

In [ ]:
# Pneu Pirelli Evo 205/55 R16 91V
insert_pneu(1, 'Formula Evo', 205, 55, 'R', 16, 91, 'V', conexao)

Inclusão de outros pneus no banco de dados.

In [ ]:
insert_pneu(1, 'Formula Evo', 195, 65, 'R', 15, 91, 'H', conexao)
insert_pneu(1, 'Formula Evo', 175, 70, 'R', 14, 84, 'T', conexao)
insert_pneu(2, 'Direction Touring SL', 175, 70, 'R', 13, 82, 'T', conexao)
insert_pneu(2, 'Direction Touring XL', 175, 70, 'R', 14, 88, 'T', conexao)
insert_pneu(2, 'Assurance MaxLife XL', 175, 65, 'R', 14, 86, 'H', conexao)
insert_pneu(5, 'Extreme Contact DW', 205, 55, 'R', 16, 91, 'W', conexao)
insert_pneu(5, 'Barum Bravuris 5HM', 175, 70, 'R', 14, 82, 'T', conexao)
insert_pneu(1, 'Cinturato P1 Plus', 205, 55, 'R', 16, 91, 'V', conexao)
insert_pneu(4, 'F700', 195, 55, 'R', 15, 85, 'H', conexao)
insert_pneu(4, 'F700', 185, 70, 'R', 14, 88, 'T', conexao)
insert_pneu(4, 'F700', 175, 70, 'R', 14, 88, 'T', conexao)
insert_pneu(3, 'Energy XM2', 195, 60, 'R', 15, 88, 'V', conexao)
insert_pneu(3, 'Energy XM2', 175, 70, 'R', 13, 82, 'T', conexao)

Na seção que segue construiremos uma função para a consulta dos modelos de pneus cadastrados no banco de dados <tt>Pneu</tt>.

## Consulta de Modelos de Pneus

A consulta dos modelos de pneus no banco de dados <tt>Pneu</tt> é feita como antes, com uso de uma função que toma, opcionalmente, uma conexão para o banco de dados ou um filtro (expressão condicional) para redução dos registros consultados. 

In [ ]:
def query_pneu(connection=None, filter=None):
    '''Função que efetua e exibe modelos de pneus cadastrados.'''
    # Obtém apenas cursor
    _, cursor = get_cursor('db/pneu.db', connection)
    # SQL para consulta de modelos de pneus
    sql = '''SELECT P.id, F.Nome, P.Modelo, P.Banda, P.Altura, P.Tipo, 
    P.Diametro, P.Carga, P.Velocidade
    FROM Fab_Pneu F
    JOIN Pneu P ON F.Id = P.Fab_Id'''
    if filter != None:
        sql += '\nWHERE ' + filter
    sql += '\nORDER BY F.Nome, P.Modelo, P.Banda, P.Altura, P.Diametro'
    # Efetua consulta
    cursor.execute(sql)
    # Dados da consulta realizada devem ser recuperados
    dados = cursor.fetchall()
    # Exibição dos dados consultados de maneira tabular
    print('+-%83s-+' % (83*'-'))
    print('| %-83s |' % ('PNEU'))
    print('+-%83s-+' % (83*'-'))
    print('| %-3s | %-15s | %-25s | %-3s | %-3s | %-2s | %-3s | %-3s | %-2s |' 
          % ('Id', 'Fabricante', 'Modelo', 'Bnd', 'Alt', 'Tp', 'Dia', 'Car', 'Vl'))
    print('+-%83s-+' % (83*'-'))
    for registro in dados:
        print('| %3d | %-15s | %-25s | %3d | %3d | %-2s | %3d | %3d | %-2s |' % registro)
    print('+-%83s-+' % (83*'-'))
    print(len(dados), 'registros.')
    return

Como antes, o uso da função <tt>query_pneu</tt> é direto.

In [ ]:
query_pneu(conexao)

Mas também permite a inclusão de um filtro, que pode envolver as colunas *Nome* (do fabricante), *Banda*, *Altura*, *Tipo*, *Diametro*, *Carga* ou *Velocidade*, como segue.

In [ ]:
query_pneu(conexao, "Banda=205 and Velocidade>='V'")

## Inclusão de Aplicação de Pneus

A inclusão da aplicação dos modelos de pneus no banco de dados <tt>Pneu</tt> requer alguns dados: id do fabricante do veículo, modelo do veículo, ano do veículo e id do pneu compatível. Como feito anteriormente, uma função pode tomar estes parâmetros e, opcionalmente, uma conexão para o banco de dados para efetuar a inclusão da aplicação de um modelo de pneu.

In [ ]:
def insert_aplicacao(fab_veic_id, modelo, ano, pneu_id, connection=None):
    '''Efetua a inclusão de aplicação de um modelo de pneu na tabela Aplicacao.'''
    # Obtém conexão e cursor
    connection, cursor = get_cursor('db/pneu.db', connection)
    # SQL para inserção de aplicação de um modelo de pneu
    sql = f"""INSERT INTO 
    Aplicacao(Fab_Veic_Id, Modelo, Ano, Pneu_Id) 
    VALUES({fab_veic_id}, '{modelo}', {ano}, {pneu_id})"""
    # Efetua inserção
    cursor.execute(sql)
    # SQL DTL (consolidação de operações)
    connection.commit()

O uso desta função é simples, como exemplificado a seguir.

In [ ]:
# Aplicação: Fiat Linea HLX 2009 --> 7, 9 e 1
insert_aplicacao(2, 'Linea', 2009, 1)
insert_aplicacao(2, 'Linea', 2009, 7)
insert_aplicacao(2, 'Linea', 2009, 9)

Inclusão de outras aplicações.

In [ ]:
insert_aplicacao(3, 'Astra', 2014, 11)
insert_aplicacao(3, 'Astra', 2013, 11)
insert_aplicacao(3, 'Vectra', 2009, 11)
insert_aplicacao(3, 'Vectra', 2010, 11)
insert_aplicacao(3, 'Cobalt', 2016, 10)
insert_aplicacao(3, 'Cobalt', 2016, 2)
insert_aplicacao(1, 'Voyage Novo', 2017, 8)
insert_aplicacao(4, 'Courrier', 2014, 4)
insert_aplicacao(2, 'Doblo', 2017, 8)
insert_aplicacao(2, 'Palio', 2017, 4)
insert_aplicacao(2, 'Doblo', 2017, 5)
insert_aplicacao(2, 'Palio', 2017, 14)
insert_aplicacao(2, 'Idea', 2017, 13)
insert_aplicacao(2, 'Punto', 2016, 13)
insert_aplicacao(6, 'Corolla', 2016, 13)
insert_aplicacao(1, 'Jetta', 2019, 1)
insert_aplicacao(1, 'Jetta', 2019, 7)
insert_aplicacao(1, 'Jetta', 2019, 9)

## Consulta de Aplicação de Pneus

A consulta das aplicações dos modelos de pneus no banco de dados <tt>Pneu</tt> usa a mesma estratégia da da função <tt>query_pneu</tt>, constituindo uma função que toma, opcionalmente, uma conexão para o banco de dados ou um filtro (expressão condicional) para redução dos registros consultados. 

In [ ]:
def query_aplicacao(connection=None, filter=None):
    '''Função que efetua e exibe aplicacoes cadastradas dos modelos de pneus.'''
    # Obtém apenas cursor
    _, cursor = get_cursor('db/pneu.db', connection)
    # SQL para consulta de aplicacoes dos modelos de pneus
    sql = '''SELECT A.id, V.Nome, A.Modelo, A.Ano, F.Nome, P.Banda,
    P.Altura, P.Tipo, P.Diametro, P.Carga, P.Velocidade
    FROM Aplicacao A
    JOIN Fab_Veic V ON A.Fab_Veic_Id = V.Id
    JOIN Pneu P ON A.Pneu_Id = P.Id
    JOIN Fab_Pneu F ON P.Fab_Id = F.Id'''
    if filter != None:
        sql += '\nWHERE ' + filter
    sql += '\nORDER BY V.Nome, A.Modelo, A.Ano, F.Nome, P.Banda, P.Altura, P.Diametro'
    # Efetua consulta
    cursor.execute(sql)
    # Dados da consulta realizada devem ser recuperados
    dados = cursor.fetchall()
    # Exibição dos dados consultados de maneira tabular
    print('+-%87s-+' % (87*'-'))
    print('| %-87s |' % ('Aplicacao'))
    print('+-%87s-+' % (87*'-'))
    print('| %-3s | %-12s | %-12s | %-4s | %-12s | %-3s | %-2s | %-2s | %-2s | %-3s | %-2s |' 
          % ('Id', 'Fabricante', 'Modelo', 'Ano', 'Pneu', 'Bnd',
             '%', 'Tp', 'Dm', 'Car', 'Vl'))
    print('+-%87s-+' % (87*'-'))
    for registro in dados:
        print('| %3d | %-12s | %-12s | %4d | %-12s | %3d | %2d | %-2s | %2d | %3d | %-2s |'
              % registro)
    print('+-%87s-+' % (87*'-'))
    print(len(dados), 'registros.')
    return

O uso da função <tt>query_aplicacao</tt> é simples e permite obter todas as aplicações cadastradas de pneus.

In [ ]:
query_aplicacao(conexao)

Outra maneira de usar da função <tt>query_aplicacao</tt> é indicando filtros para restringir os resultados da consulta. Os filtros podem combinar as colunas: *V.Nome* (do fabricante de veículos), *A.Modelo* (de veículo), *Ano* (do veículo), *F.Nome* (do fabricante de pneus), *P.Modelo* (do pneu); além de outros dados do pneu: *Banda*, *Altura*, *Tipo*, *Diametro*, *Carga* e *Velocidade*

In [ ]:
query_aplicacao(conexao, "A.Modelo='Doblo'")

O uso das funções descritas neste *notebook* permitem construir uma aplicação completa, embora simples, para manter um cadastro útil de aplicações de pneus para automóveis de passeio ou carga.

### FIM
### <a href="http://github.com/pjandl/opy2">Oficina Python Intermediário</a>